# Helpers

> Helper functions for BYOK system

In [ ]:
#| default_exp utils.helpers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional, Any
from datetime import datetime, timedelta
import re

from cjm_fasthtml_byok.core.security import mask_key, get_key_fingerprint

## Provider Helpers

In [ ]:
#| export
# Common provider configurations
PROVIDER_INFO = {
    'openai': {
        'name': 'OpenAI',
        'key_prefix': 'sk-',
        'docs_url': 'https://platform.openai.com/api-keys',
        'placeholder': 'sk-...'
    },
    'anthropic': {
        'name': 'Anthropic',
        'key_prefix': 'sk-ant-',
        'docs_url': 'https://console.anthropic.com/settings/keys',
        'placeholder': 'sk-ant-...'
    },
    'google': {
        'name': 'Google Gemini',
        'key_prefix': None,
        'docs_url': 'https://makersuite.google.com/app/apikey',
        'placeholder': 'Your API key'
    },
    'fireworks': {
        'name': 'Fireworks AI',
        'key_prefix': None,
        'docs_url': 'https://app.fireworks.ai/api-keys',
        'placeholder': 'Your API key'
    },
    'groq': {
        'name': 'Groq',
        'key_prefix': 'gsk_',
        'docs_url': 'https://console.groq.com/keys',
        'placeholder': 'gsk_...'
    },
    'xai': {
        'name': 'xAI Grok',
        'key_prefix': 'xai-',
        'docs_url': 'https://console.x.ai/',
        'placeholder': 'xai-...'
    },
}

def get_provider_info(
    provider: str  # Provider identifier
) -> Dict[str, Any]:  # Provider info dict or default values
    "Get provider information."
    return PROVIDER_INFO.get(provider, {
        'name': provider.title(),
        'key_prefix': None,
        'docs_url': None,
        'placeholder': 'Your API key'
    })

## Format Helpers

In [ ]:
#| export
def format_provider_name(
    provider: str  # Provider identifier
) -> str:  # Formatted provider name
    "Format provider name for display."
    info = get_provider_info(provider)
    return info['name']

In [ ]:
#| export
def format_key_age(
    created_at: datetime  # When the key was created
) -> str:  # Human-readable age string
    "Format the age of a key for display."
    age = datetime.now() - created_at
    
    if age.days > 30:
        months = age.days // 30
        return f"{months} month{'s' if months > 1 else ''} ago"
    elif age.days > 0:
        return f"{age.days} day{'s' if age.days > 1 else ''} ago"
    elif age.seconds > 3600:
        hours = age.seconds // 3600
        return f"{hours} hour{'s' if hours > 1 else ''} ago"
    elif age.seconds > 60:
        minutes = age.seconds // 60
        return f"{minutes} minute{'s' if minutes > 1 else ''} ago"
    else:
        return "Just now"

In [ ]:
#| export
def format_expiration(
    expires_at: Optional[datetime]  # Expiration datetime
) -> str:  # Human-readable expiration string
    "Format expiration time for display."
    if not expires_at:
        return "Never"
    
    if expires_at <= datetime.now():
        return "Expired"
    
    remaining = expires_at - datetime.now()
    
    if remaining.days > 0:
        return f"Expires in {remaining.days} day{'s' if remaining.days > 1 else ''}"
    elif remaining.seconds > 3600:
        hours = remaining.seconds // 3600
        return f"Expires in {hours} hour{'s' if hours > 1 else ''}"
    else:
        minutes = remaining.seconds // 60
        return f"Expires in {minutes} minute{'s' if minutes > 1 else ''}"

## Key Summary

In [ ]:
#| export
def get_key_summary(
    byok_manager,  # BYOK manager instance
    request,  # FastHTML request
    user_id: Optional[str] = None  # Optional user ID
) -> Dict[str, Any]:  # Summary dictionary with provider info
    "Get a summary of all stored keys."
    providers = byok_manager.list_providers(request, user_id)
    summary = []
    
    for provider in providers:
        # Get the key metadata (without decrypting)
        key_obj = byok_manager.storage.retrieve(request, provider, user_id)
        if key_obj:
            # Get the actual key for masking
            actual_key = byok_manager.get_key(request, provider, user_id)
            
            summary.append({
                'provider': provider,
                'display_name': format_provider_name(provider),
                'masked_key': mask_key(actual_key) if actual_key else 'N/A',
                'fingerprint': get_key_fingerprint(actual_key) if actual_key else 'N/A',
                'created': format_key_age(key_obj.created_at),
                'expires': format_expiration(key_obj.expires_at),
                'is_expired': key_obj.is_expired
            })
    
    return {
        'count': len(summary),
        'keys': summary
    }

## Environment Helpers

In [ ]:
#| export
def get_env_key(
    provider: str,  # Provider name
    env_prefix: str = "API_KEY_"  # Environment variable prefix
) -> Optional[str]:  # API key from environment or None
    "Get an API key from environment variables."
    import os
    env_var = f"{env_prefix}{provider.upper()}"
    return os.environ.get(env_var)

In [ ]:
#| export
def import_from_env(
    byok_manager,  # BYOK manager instance
    request,  # FastHTML request
    providers: List[str],  # List of providers to check
    user_id: Optional[str] = None,  # Optional user ID
    env_prefix: str = "API_KEY_"  # Environment variable prefix
) -> Dict[str, bool]:  # Dict of provider: success status
    "Import API keys from environment variables."
    results = {}
    
    for provider in providers:
        key = get_env_key(provider, env_prefix)
        if key:
            try:
                byok_manager.set_key(request, provider, key, user_id)
                results[provider] = True
            except Exception:
                results[provider] = False
        else:
            results[provider] = False
    
    return results

## Tests

In [ ]:
# Test provider info
info = get_provider_info('openai')
assert info['name'] == 'OpenAI'
assert info['key_prefix'] == 'sk-'
print(f"✓ Provider info: {info['name']}")

# Test formatting
assert format_provider_name('openai') == 'OpenAI'
assert format_provider_name('unknown') == 'Unknown'
print("✓ Provider name formatting")

# Test age formatting
now = datetime.now()
assert format_key_age(now) == "Just now"
assert format_key_age(now - timedelta(hours=2)) == "2 hours ago"
assert format_key_age(now - timedelta(days=5)) == "5 days ago"
print("✓ Age formatting")

# Test expiration formatting
assert format_expiration(None) == "Never"
assert format_expiration(now - timedelta(hours=1)) == "Expired"
assert "Expires in" in format_expiration(now + timedelta(hours=5))
print("✓ Expiration formatting")

✓ Provider info: OpenAI
✓ Provider name formatting
✓ Age formatting
✓ Expiration formatting


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()